Ce script agrège les données issues des déclarations des organismes à partir d'un fichier excel possédant plusieurs onglets. L'emplacement des données est donnée par les variables suivantes :
- **`DATA_DIR`** : remplacement du dossier des données relativement aux scripts python ('..' fait référence au dossier parent)
- **`EXCEL_ROAM_BRUT`** : nom du fichier Excel contenant les déclarations des organismes. Ce fichier a été légèrement retouché : il doit comporter la colonne "Code SAS" ainsi que la colonne "Typologie" qui a été pré-renseignée (on verra qu'un deuxième passage sur cette colonne sera nécessaire pour corriger certaines typologies, qualifiées comme relevant de TRAM à tort)
- **`ONGLET_MUT_CLSQ`** : onglet du fichier excel contenant les déclarations relatives aux "mutualisations classiques" (typologies "Mutualisation", hors TRAM, et "Pole régionaux", TRAM)
- **`ONGLET_MUT_PHR`** : onglet du fichier excel contenant les déclarations relatives à "PHARE" (typologie du même nom ; cet onglet doit avoir exactement la même structure que ONGLET_MUT_CLSQ)
- **`ONGLET_MUT_NAT`** : onglet du fichier excel contenant les déclarations relatives aux "Centres Nationaux" (typologie du même nom ; cet onglet doit avoir exactement la même structure que ONGLET_MUT_CLSQ)
- **`ONGLET_DEDUC`** : onglet du fichier excel contenant les déclarations relatives aux déductions hors mutualisation (comprenant donc toutes les autres mutualisations ; cet onglet se présente légèrement différemment des autres)

# Import des librairies et variables

- la librairie **`os`** est utile pour naviguer dans les fichiers, préciser les chemins
- la librairie **`numpy`** est utile pour les calculs numériques, ici nous l'utilisons pour les données manquantes (NAN : not a number avec numpy.nan)
- la librairie **`pandas`** est utile pour gérer des données tabulaires

On importe ensuite les variables décrites au-dessus

In [1]:
import os

import numpy as np
import pandas as pd

from config_variables_globales import DATA_DIR, EXCEL_ROAM_BRUT, ONGLET_MUT_CLSQ, ONGLET_MUT_PHR, ONGLET_MUT_NAT, ONGLET_DEDUC

# Chargement des déclarations ROAM

La fonction **`read_xls_deduc_tab`** est dédiée à la lecture des données relatives aux déductions hors mutualisations. Les principales étapes sont les suivantes :
1. lecture de l'onglet excel en précisant le format **`str`** ("string") des différentes colonnes, en particulier celles relatives aux codes organismes
2. renommage des colonnes pour se conformer à un format unique entre les différents onglets
3. création d'un DataFrame (un tableau) vide, mais comportant les colonnes nécessaires pour se conformer au format unique, et concaténation avec les données relatives aux déductions hors mutualisations (via l'opération **`pd.concat`** (concaténation sur les colonnes, en précisant l'argument **`axis=1`**)

In [2]:
# chargement des données relatives aux déductions hors mutualisation
def read_xls_deduc_tab(onglet_deduc, data_dir=DATA_DIR, excel_roam_brut=EXCEL_ROAM_BRUT):
    deduc_data = pd.read_excel(io=os.path.join(data_dir, excel_roam_brut),
                               sheetname=onglet_deduc,
                               converters={'Code Caisse': str,
                                           'Caisse': str,
                                           'Région': str,
                                           'Code  DI (96)': str,
                                           'Code SAS': str,
                                           'Typologie': str,
                                           'Préciser obligatoirement le motif  ': str})

    # renommage des colonnes
    deduc_data.rename(index=str,
                      columns={'Code Caisse': 'Code Caisse prenante',
                               'Caisse': 'Caisse prenante',
                               'Code  DI (96)': 'Processus',
                               'Préciser obligatoirement le motif  ': 'Thème de mutualisation',
                               'ETP réellement déduits ': 'ETP à déduire',
                               'Total charges réellement déduites': 'Total charges à déduire'},
                      inplace=True)

    # rajout des colonnes manquantes (NaN pour les vides, "Pas une mutualisation" pour le périmètre)
    nan_list = [np.nan] * deduc_data.shape[0]
    df_empty_columns = pd.DataFrame({'Code Caisse': nan_list,
                                     'Caisse': nan_list,
                                     'ETP à ajouter': nan_list,
                                     'Total charges à ajouter': nan_list,
                                     'La mutualisation est-elle totale ou partielle?': ['Pas une mutualisation'] * deduc_data.shape[0]})
    df_empty_columns.index = deduc_data.index
    deduc_data = pd.concat([deduc_data, df_empty_columns], axis=1)

    return deduc_data[['Code Caisse prenante',
                       'Caisse prenante',
                       'Code Caisse',
                       'Caisse',
                       'La mutualisation est-elle totale ou partielle?',
                       'Processus',
                       'Code SAS',
                       'Thème de mutualisation',
                       'Typologie',
                       'ETP à ajouter',
                       'ETP à déduire',
                       'Total charges à ajouter',
                       'Total charges à déduire']]

La fonction **`read_xls_mut_tab`** est dédiée à la lecture des données relatives aux mutualisations au sens large (onglets **`ONGLET_MUT_CLSQ`**, **`ONGLET_MUT_PHR`** et **`ONGLET_MUT_NAT`**).

Les onglets sont quasiment conservés dans leur état natif. On précise simplement le format sous lequel on souhaite lire les différentes colonnes (en particulier, format **`str`** pour les colonnes relatives aux codes organisme).

In [3]:
# chargement des données relatives aux mutualisations ("mutualisations classiques", "centres nationaux" et PHARE")
def read_xls_mut_tab(onglet_mut, data_dir=DATA_DIR, excel_roam_brut=EXCEL_ROAM_BRUT):
    mut_data = pd.read_excel(io=os.path.join(data_dir, excel_roam_brut),
                             sheetname=onglet_mut,
                             converters={'Code Caisse': str,
                                         'Caisse': str,
                                         'Région': str,
                                         'Processus': str,
                                         'Code SAS': str,
                                         'Typologie': str,
                                         'Thème de mutualisation': str,
                                         'La mutualisation est-elle totale ou partielle?': str,
                                         'Code Caisse prenante': str,
                                         'Caisse prenante': str,
                                         'Région.1': str})

    return mut_data[['Code Caisse prenante',
                     'Caisse prenante',
                     'Code Caisse',
                     'Caisse',
                     'La mutualisation est-elle totale ou partielle?',
                     'Processus',
                     'Code SAS',
                     'Thème de mutualisation',
                     'Typologie',
                     'ETP à ajouter',
                     'ETP à déduire',
                     'Total charges à ajouter',
                     'Total charges à déduire']]

La fonction **`process_data`** retraite le dataframe obtenu après lecture des données déclarées par les organismes.

Les traitements sont les suivants :
- renommage des colonnes
- ajout de la colonne statut après identification des statuts cédant / prenant :
    - identification des lignes pour lesquelles le statut est cédant (lignes pour lesquelles les charges à déduire mais aussi les ETP à déduire sont nulles)
    - identification des lignes pour lesquelles le statut st prenante (situation inverse : toutes les lignes pour lesquelles soit les charges à déduire et/ou les ETP à déduire ne sont pas nulles)
- ajout du libellé organisme quand le code n'est pas renseigné (cas de figures concernés : "AUTRES", "AUTRE - CARSAT", "AUTREs - ELSM/DRSM")
- calcul des colonnes unifiées "charges à déduire" ou "ETP à déduire" (avec le signe + ou - selon le statut cédant ou prenant)

In [4]:
# chargement des données relatives aux mutualisations "classiques"
def process_data(data):
    # extraction du code analytique dans la colonne "Processus"
    data['Processus'] = data['Processus'].apply(lambda row: row[:4])

    # renommage des colonnes
    data.rename(index=str,
                columns={'Code Caisse prenante': 'Code organisme prenant',
                         'Caisse prenante': 'Organisme prenant',
                         'Code Caisse': 'Code organisme cédant',
                         'Caisse': 'Organisme cédant',
                         'La mutualisation est-elle totale ou partielle?': 'Périmètre',
                         'Thème de mutualisation': 'Motif',
                         'Processus': 'Code analytique'},
                inplace=True)

    # ajout de la colonne "Statut"
    statut_c = data.apply(func=lambda row: np.int(np.isnan(row['Total charges à déduire']) & np.isnan(row['ETP à déduire'])) * "Cédant", axis=1)
    statut_p = data.apply(func=lambda row: (1 - np.int(np.isnan(row['Total charges à déduire']) & np.isnan(row['ETP à déduire']))) * "Prenant", axis=1)
    statut = statut_c + statut_p
    statut.name = "Statut"
    data = pd.concat([data, statut], axis=1)

    # ajout du libellé organisme quand le code n'est pas renseigné (cas de figure : "AUTRES", "AUTRE - CARSAT", "AUTREs - ELSM/DRSM")
    data.loc[data["Code organisme prenant"].isnull(), 'Code organisme prenant'] = data.loc[data["Code organisme prenant"].isnull(), 'Organisme prenant']
    data.loc[data["Code organisme cédant"].isnull(), 'Code organisme cédant'] = data.loc[data["Code organisme cédant"].isnull(), 'Organisme cédant']

    # ajout de la colonne "ETP à déduire/ajouter"
    data['ETP à déduire/ajouter'] = data['ETP à déduire']

    data.loc[data['Statut'] == 'Cédant', 'ETP à déduire/ajouter'] = -1 * data.loc[data['Statut'] == 'Cédant', 'ETP à ajouter']
    # ajout de la colonne "Charges à déduire/ajouter"
    data['Charges à déduire/ajouter'] = data['Total charges à déduire']
    data.loc[data['Statut'] == 'Cédant', 'Charges à déduire/ajouter'] = -1 * data.loc[data['Statut'] == 'Cédant', 'Total charges à ajouter']

    # suppression des colonnes inutiles et ré-ordonnancement des colonnes
    data = data[["Organisme prenant",
                 "Code organisme prenant",
                 "Organisme cédant",
                 "Code organisme cédant",
                 "Statut",
                 "Périmètre",
                 "Code analytique",
                 "Code SAS",
                 "Motif",
                 "Typologie",
                 'ETP à déduire/ajouter',
                 'Charges à déduire/ajouter']]

    return data

La fonction finale **`etl_roam`** applique séquentiellement la lecture des déductions (mutualisations et déductions hors mutualisations) ainsi que leur retraitement. Les données sont ensuite concaténées en un tableau final (l'argument `axis=0` précise que la concaténation se fai au niveau des lignes, et non des colonnes).

In [5]:
def etl_roam(data_dir=DATA_DIR, excel_roam_brut=EXCEL_ROAM_BRUT, onglet_mut_clsq=ONGLET_MUT_CLSQ, onglet_mut_phr=ONGLET_MUT_PHR, onglet_mut_nat=ONGLET_MUT_NAT, onglet_deduc=ONGLET_DEDUC):
    mut_clsq_data = process_data(read_xls_mut_tab(onglet_mut=onglet_mut_clsq, data_dir=data_dir, excel_roam_brut=excel_roam_brut))
    mut_phr_data = process_data(read_xls_mut_tab(onglet_mut=onglet_mut_phr, data_dir=data_dir, excel_roam_brut=excel_roam_brut))
    mut_nat_data = process_data(read_xls_mut_tab(onglet_mut=onglet_mut_nat, data_dir=data_dir, excel_roam_brut=excel_roam_brut))
    deduc_data = process_data(read_xls_deduc_tab(onglet_deduc=onglet_deduc, data_dir=data_dir, excel_roam_brut=excel_roam_brut))
    data = pd.concat([mut_clsq_data, mut_phr_data, mut_nat_data, deduc_data], axis=0)
    #     data = add_ETP_ana(data)
    return data

# data processing

In [6]:
if __name__ == '__main__':
    data = etl_roam()
    writer = pd.ExcelWriter(os.path.join(DATA_DIR, 'ROAM_2016.xlsx'))
    data.to_excel(excel_writer=writer, sheet_name='ROAM 2016', index=False)
    writer.save()